In [1]:
import os

from utility_dot_py.utility import QueryProcessor
from assistments_workbench.data_access.db_access.connector import db
from assistments_workbench.data_access.attach_query_result import attach_query_result_by_header
from assistments_workbench.config_reader import config

In [2]:
# define output folder
output_dir = config.get('localfiles', 'data_path')

In [3]:
# sql query for getting user id, problem id, problem set id, and correctness
sql_query = """ select user_id, sequence_id, problem_id, correct from problem_logs pl
left join class_assignments ca on ca.id = pl.assignment_id
where ca.sequence_id in (
5968,11898,6921,5969,37570
) and pl.start_time::date >= '2015-11-01' and pl.start_time::date < '2015-12-01' and pl.correct is not null
and pl.original = 1
order by user_id, pl.id; """

In [4]:
processor = QueryProcessor(db, output_dir)

In [5]:
sql_data_path = processor.run_query_to_csv(sql_query)

Running queries:   select user_id, sequence_id, problem_id, correct from problem_logs pl
left join class_assignments ca on ca.id = pl.assignment_id
where ca.sequence_id in (
5968,11898,6921,5969,37570
) and pl.start_time::date >= '2015-11-01' and pl.start_time::date < '2015-12-01' and pl.correct is not null
and pl.original = 1
order by user_id, pl.id; 
C:\Workarea\Data\query_result_2016_04_02_22_43_32.csv


In [ ]:
print sql_data_path

C:\Workarea\Data\query_result_2016_04_02_22_43_32.csv


In [ ]:
sql_with_features = attach_query_result_by_header(sql_data_path)

contained header:  [('problem_id',)]
new header:  ['user_id', 'sequence_id', 'problem_id', 'correct', 'problem_difficulty']
Running queries to attach new information ...


15it [00:49,  2.35s/it]

In [ ]:
print sql_with_features